# Data Visualisation | Homogenous Transforms
- The following is code that uses traffic data captured by LIDAR and camera input to produce 3D visualisations of the cars present in that data
- The code includes very minor refactoring, but is majoritively based off of this Kaggle notebook (https://www.kaggle.com/code/sakshaymahna/lidar-data-visualization/notebook) as part of this tutorial (https://www.youtube.com/watch?v=cPOtULagNnI)

## 1. Dataset Functions
Functions to load LIDAR data

In [ ]:
import os
import cv2
import re
import numpy as np
import matplotlib.pyplot as plt
import vtk
from vedo import *
import random
import glob
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
# Function to read calibration file
# Input: Calibration Text File Path
# Output: P2: 3D camera coordinates to 2D image pixels
# Output: vtc_mat: 3D Lidar coordinates to 3D camera coordinates 
def read_calib(calib_path):
    with open(calib_path) as f:
        for line in f.readlines():
            if line[:2] == "P2":
                P2 = re.split(" ", line.strip())
                P2 = np.array(P2[-12:], np.float32)
                P2 = P2.reshape((3, 4))
            if line[:14] == "Tr_velo_to_cam" or line[:11] == "Tr_velo_cam":
                vtc_mat = re.split(" ", line.strip())
                vtc_mat = np.array(vtc_mat[-12:], np.float32)
                vtc_mat = vtc_mat.reshape((3, 4))
                vtc_mat = np.concatenate([vtc_mat, [[0, 0, 0, 1]]])
            if line[:7] == "R0_rect" or line[:6] == "R_rect":
                R0 = re.split(" ", line.strip())
                R0 = np.array(R0[-9:], np.float32)
                R0 = R0.reshape((3, 3))
                R0 = np.concatenate([R0, [[0], [0], [0]]], -1)
                R0 = np.concatenate([R0, [[0, 0, 0, 1]]])
    vtc_mat = np.matmul(R0, vtc_mat)
    return (P2, vtc_mat)

# Function to read lidar data
# Input: Path to lidar bin file
# Input: Camera 3D to Camera 2D Matrix
# Input: Lidar 3D to Camera 3D Matrix
# Output: Valid points in Lidar Coordinates
def read_velodyne(path, P, vtc_mat, IfReduce=True):
    max_row = 374  # y
    max_col = 1241  # x
    lidar = np.fromfile(path, dtype=np.float32).reshape((-1, 4))

    if not IfReduce:
        return lidar

    mask = lidar[:, 0] > 0
    lidar = lidar[mask]
    lidar_copy = np.zeros(shape=lidar.shape)
    lidar_copy[:, :] = lidar[:, :]

    velo_tocam = vtc_mat
    lidar[:, 3] = 1
    lidar = np.matmul(lidar, velo_tocam.T)
    img_pts = np.matmul(lidar, P.T)
    velo_tocam = np.mat(velo_tocam).I
    velo_tocam = np.array(velo_tocam)
    normal = velo_tocam
    normal = normal[0:3, 0:4]
    lidar = np.matmul(lidar, normal.T)
    lidar_copy[:, 0:3] = lidar
    x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]
    mask = np.logical_and(np.logical_and(x >= 0, x < max_col), np.logical_and(y >= 0, y < max_row))

    return lidar_copy[mask]

# Function to convert 3D Camera coordinates to 3D Lidar coordinates
# Input: 3D Camera Points
# Input: Lidar 3D to Camera 3D Matrix
# Output: 3D Lidar Points

def cam_to_velo(cloud,vtc_mat):
    mat=np.ones(shape=(cloud.shape[0],4),dtype=np.float32)
    mat[:,0:3]=cloud[:,0:3]
    mat=np.mat(mat)
    normal=np.mat(vtc_mat).I
    normal=normal[0:3,0:4]
    transformed_mat = normal * mat.T
    T=np.array(transformed_mat.T,dtype=np.float32)
    return T

# Function to convert 3D Lidar coordinates to 3D Camera coordinates
# Input: 3D Camera Points
# Input: Lidar 3D to Camera 3D Matrix
# Output: 3D Lidar Points
def velo_to_cam(cloud,vtc_mat):
    mat=np.ones(shape=(cloud.shape[0],4),dtype=np.float32)
    mat[:,0:3]=cloud[:,0:3]
    mat=np.mat(mat)
    normal=np.mat(vtc_mat)
    transformed_mat = normal * mat.T
    T=np.array(transformed_mat.T,dtype=np.float32)
    return T

# Function to read image
# Input: Image path
# Output: Image matrix
def read_image(path):
    im=cv2.imdecode(np.fromfile(path, dtype=np.uint8), -1)
    return im

# Function to read labels
# Input: Input label path file
# Output: Array of Box coordinates
# Output: Array of label names
def read_detection_label(path):

    boxes = []
    names = []

    with open(path) as f:
        for line in f.readlines():
            line = line.split()
            this_name = line[0]
            if this_name != "DontCare":
                line = np.array(line[-7:],np.float32)
                boxes.append(line)
                names.append(this_name)

    return np.array(boxes),np.array(names)

# Function to read tracking label
# Input: Input label path file
# Output: Frame Dictionary
# Output: Frame Name dictionary
def read_tracking_label(path):

    frame_dict={}

    names_dict={}

    with open(path) as f:
        for line in f.readlines():
            line = line.split()
            this_name = line[2]
            frame_id = int(line[0])
            ob_id = int(line[1])

            if this_name != "DontCare":
                line = np.array(line[10:17],np.float32).tolist()
                line.append(ob_id)


                if frame_id in frame_dict.keys():
                    frame_dict[frame_id].append(line)
                    names_dict[frame_id].append(this_name)
                else:
                    frame_dict[frame_id] = [line]
                    names_dict[frame_id] = [this_name]

    return frame_dict,names_dict

## 2. Dataset Functions

In [ ]:
# Detection Dataset Class
class KittiDetectionDataset:
    # Initialization Function to read all paths
    def __init__(self,root_path,label_path = None):
        self.root_path = root_path
        self.velo_path = os.path.join(self.root_path,"velodyne")
        self.image_path = os.path.join(self.root_path,"image_2")
        self.calib_path = os.path.join(self.root_path,"calib")
        if label_path is None:
            self.label_path = os.path.join(self.root_path, "label_2")
        else:
            self.label_path = label_path

        self.all_ids = os.listdir(self.velo_path)

    # Length Function
    def __len__(self):
        return len(self.all_ids)
    
    # Get index function
    def __getitem__(self, item):
        name = str(item).zfill(6)

        velo_path = os.path.join(self.velo_path,name+'.bin')
        image_path = os.path.join(self.image_path, name+'.png')
        calib_path = os.path.join(self.calib_path, name+'.txt')
        label_path = os.path.join(self.label_path, name+".txt")

        P2,V2C = read_calib(calib_path)
        points = read_velodyne(velo_path,P2,V2C)
        image = read_image(image_path)
        labels,label_names = read_detection_label(label_path)
        labels[:,3:6] = cam_to_velo(labels[:,3:6],V2C)[:,:3]

        return P2,V2C,points,image,labels,label_names

## 3. Utility Functions
Utility functions for visualisation

In [ ]:
# Download dependency
!pip install vedo

# Download object files for Cars and Ego Car
!wget https://github.com/hailanyi/3D-Detection-Tracking-Viewer/blob/master/viewer/car.obj
!wget https://github.com/hailanyi/3D-Detection-Tracking-Viewer/blob/master/viewer/ego_car.3ds

In [ ]:
# Function to convert the input box to a common type
# Input: Input boxes
# Output: New boxes as [x, y, z, l, w, h, yaw]
def convert_box_type(boxes,input_box_type = 'Kitti'):
    boxes = np.array(boxes)
    if len(boxes) == 0:
        return None
    assert  input_box_type in ["Kitti","OpenPCDet","Waymo"], 'unsupported input box type!'

    if input_box_type in ["OpenPCDet","Waymo"]:
        return boxesdfsa

    if input_box_type == "Kitti": #(h,w,l,x,y,z,yaw) -> (x,y,z,l,w,h,yaw)
        boxes = np.array(boxes)
        new_boxes = np.zeros(shape=boxes.shape)
        new_boxes[:,:]=boxes[:,:]
        new_boxes[:,0:3] = boxes[:,3:6]
        new_boxes[:, 3] = boxes[:, 2]
        new_boxes[:, 4] = boxes[:, 1]
        new_boxes[:, 5] = boxes[:, 0]
        new_boxes[:, 6] = (np.pi - boxes[:, 6]) + np.pi / 2
        new_boxes[:, 2] += boxes[:, 0] / 2
        return new_boxes

# Convert box array to mesh box
# Input: boxes array
# Output: mesh boxes array
def get_mesh_boxes(boxes,colors="red",
                   mesh_alpha=0.4,
                   ids=None,
                   show_ids=False,
                   box_info=None,
                   show_box_info=False,
                   caption_size=(0.05,0.05)):
    vtk_boxes_list = []
    for i in range(len(boxes)):
        box = boxes[i]
        angle = box[6]

        new_angle = (angle / np.pi) * 180

        if type(colors) is str:
            this_c = colors
        else:
            this_c = colors[i]
        vtk_box = Box(pos=(0, 0, 0), height=box[5], width=box[4], length=box[3], c=this_c, alpha=mesh_alpha)
        vtk_box.rotateZ(new_angle)
        vtk_box.pos(box[0], box[1], box[2])

        info = ""
        if ids is not None and show_ids :
            info = "ID: "+str(ids[i])+'\n'
        if box_info is not None and show_box_info:
            info+=str(box_info[i])
        if info !='':
            vtk_box.caption(info,point=(box[0],
                            box[1]-box[4]/4, box[2]+box[5]/2),
                            size=caption_size,
                            alpha=1,c=this_c,
                            font="Calco",
                            justify='left')
            vtk_box._caption.SetBorder(False)
            vtk_box._caption.SetLeader(False)

        vtk_boxes_list.append(vtk_box)

    return vtk_boxes_list

# Function to get line boxes
# Input: boxes array
# Output: lines, arrows and spheres array
def get_line_boxes(boxes,
                   colors,
                   show_corner_spheres=True,
                   corner_spheres_alpha=1,
                   corner_spheres_radius=0.3,
                   show_heading=True,
                   heading_scale=1,
                   show_lines=True,
                   line_width=2,
                   line_alpha=1,
                   ):
    lines_actors = []
    sphere_actors = []
    arraw_actors = []


    for i in range(len(boxes)):
        box = boxes[i]
        corner_points = []
        corner_points1 = []
        corner_points2 = []
        arraw_points1 = []
        arraw_points2 = []

        angle = box[6]

        new_angle = angle

        transform_mat = np.array([[np.cos(new_angle), -np.sin(new_angle), 0, box[0]],
                                  [np.sin(new_angle), np.cos(new_angle), 0, box[1]],
                                  [0, 0, 1, box[2]],
                                  [0, 0, 0, 1]])
        x = box[3]
        y = box[4]
        z = box[5]

        corner_points.append([-x / 2, -y / 2, -z / 2, 1])
        corner_points.append([-x / 2, -y / 2, z / 2, 1])
        corner_points.append([-x / 2, y / 2, z / 2, 1])
        corner_points.append([-x / 2, y / 2, -z / 2, 1])
        corner_points.append([x / 2, y / 2, -z / 2, 1])
        corner_points.append([x / 2, y / 2, z / 2, 1])
        corner_points.append([x / 2, -y / 2, z / 2, 1])
        corner_points.append([x / 2, -y / 2, -z / 2, 1])

        corner_points1.append([-x / 2, -y / 2, - z / 2, 1])
        corner_points1.append([-x / 2, -y / 2, z / 2, 1])
        corner_points1.append([-x / 2, y / 2, z / 2, 1])
        corner_points1.append([-x / 2, y / 2, -z / 2, 1])
        corner_points1.append([-x / 2, y / 2, z / 2, 1])
        corner_points1.append([-x / 2, -y / 2, z / 2, 1])
        corner_points1.append([x / 2, -y / 2, z / 2, 1])
        corner_points1.append([x / 2, y / 2, z / 2, 1])
        corner_points1.append([-x / 2, -y / 2, -z / 2, 1])
        corner_points1.append([x / 2, -y / 2, -z / 2, 1])
        corner_points1.append([x / 2, -y / 2, z / 2, 1])
        corner_points1.append([-x / 2, -y / 2, z / 2, 1])

        corner_points2.append([x / 2, -y / 2, - z / 2, 1])
        corner_points2.append([x / 2, -y / 2, z / 2, 1])
        corner_points2.append([x / 2, y / 2, z / 2, 1])
        corner_points2.append([x / 2, y / 2, -z / 2, 1])
        corner_points2.append([-x / 2, y / 2, -z / 2, 1])
        corner_points2.append([-x / 2, -y / 2, -z / 2, 1])
        corner_points2.append([x / 2, -y / 2, -z / 2, 1])
        corner_points2.append([x / 2, y / 2, -z / 2, 1])
        corner_points2.append([-x / 2, y / 2, -z / 2, 1])
        corner_points2.append([x / 2, y / 2, -z / 2, 1])
        corner_points2.append([x / 2, y / 2, z / 2, 1])
        corner_points2.append([-x / 2, y / 2, z / 2, 1])

        arraw_points1.append([0, 0, 0, 1])
        arraw_points2.append([x / 2 , 0, 0, 1])

        corner_points = np.matmul(np.array(corner_points), transform_mat.T)
        corner_points1 = np.matmul(np.array(corner_points1), transform_mat.T)
        corner_points2 = np.matmul(np.array(corner_points2), transform_mat.T)
        arraw_points1 = np.matmul(np.array(arraw_points1), transform_mat.T)
        arraw_points2 = np.matmul(np.array(arraw_points2), transform_mat.T)

        if type(colors) is not str:
            this_c = colors[i]
            corner_colors = np.tile(this_c,(corner_points.shape[0],1))
            arraw_colors = np.tile(this_c,(arraw_points1.shape[0],1))

        else:
            this_c = colors
            corner_colors = colors
            arraw_colors = colors

        lines = Lines(corner_points1[:, 0:3], corner_points2[:, 0:3], c=this_c, alpha=line_alpha, lw=line_width)

        corner_spheres = Spheres(corner_points[:,0:3], c= corner_colors, r=corner_spheres_radius,res = 12,alpha=corner_spheres_alpha)

        arraws = Arrows(arraw_points1[:,0:3],arraw_points2[:,0:3],c = arraw_colors,s=heading_scale)
        lines_actors.append(lines)
        sphere_actors.append(corner_spheres)
        arraw_actors.append(arraws)

    return_list =[]

    if show_corner_spheres:
        return_list+=sphere_actors
    if show_heading:
        return_list+=arraw_actors
    if show_lines:
        return_list+=lines_actors

    return return_list

# Function to get transformed box points
# Input: box array
# Output: transformed matrix
def get_box_points(points, pose=None):
    PI=np.pi
    import math
    point=np.zeros(shape=points.shape)
    point[:]=points[:]

    h,w,l = point[5],point[4],point[3]
    x,y,z = point[0],point[1],point[2]


    point_num=200
    i=1
    label=1
    z_vector = np.arange(- h / 2, h / 2, h / point_num)[0:point_num]
    w_vector = np.arange(- w / 2, w / 2, w / point_num)[0:point_num]
    l_vector = np.arange(- l / 2, l / 2, l / point_num)[0:point_num]

    d_z_p = -np.sort(-np.arange(0, h / 2, h / (point_num*2))[0:point_num])
    d_z_n = np.arange( -h / 2,0, h / (point_num*2))[0:point_num]


    d_w_p = -np.sort(-np.arange(0, w / 2, w / (point_num*2))[0:point_num])
    d_w_n = np.arange(-w / 2,0,  w / (point_num*2))[0:point_num]

    d_l_p = np.arange(l / 2, l*(4/7) , (l*(4/7)-l / 2) / (point_num*2))[0:point_num]


    d1 = np.zeros(shape=(point_num, 4))
    d1[:, 0] = d_w_p
    d1[:, 1] = d_l_p
    d1[:, 2] = d_z_p
    d1[:, 3] = i

    d2 = np.zeros(shape=(point_num, 4))
    d2[:, 0] = d_w_n
    d2[:, 1] = d_l_p
    d2[:, 2] = d_z_p
    d2[:, 3] = i

    d3 = np.zeros(shape=(point_num, 4))
    d3[:, 0] = d_w_p
    d3[:, 1] = d_l_p
    d3[:, 2] = d_z_n
    d3[:, 3] = i

    d4 = np.zeros(shape=(point_num, 4))
    d4[:, 0] = d_w_n
    d4[:, 1] = d_l_p
    d4[:, 2] = d_z_n
    d4[:, 3] = i

    z1 = np.zeros(shape=(point_num, 4))
    z1[:, 0] = -w / 2
    z1[:, 1] = -l / 2
    z1[:, 2] = z_vector
    z1[:, 3] = i
    z2 = np.zeros(shape=(point_num, 4))
    z2[:, 0] = -w / 2
    z2[:, 1] =l / 2
    z2[:, 2] = z_vector
    z2[:, 3] = i
    z3 = np.zeros(shape=(point_num, 4))
    z3[:, 0] = w / 2
    z3[:, 1] = -l / 2
    z3[:, 2] = z_vector
    z3[:, 3] = i
    z4 = np.zeros(shape=(point_num, 4))
    z4[:, 0] = w / 2
    z4[:, 1] = l / 2
    z4[:, 2] = z_vector
    z4[:, 3] = i
    w1 = np.zeros(shape=(point_num, 4))
    w1[:, 0]=w_vector
    w1[:, 1]=-l / 2
    w1[:, 2]=-h / 2
    w1[:, 3] = i
    w2 = np.zeros(shape=(point_num, 4))
    w2[:, 0] = w_vector
    w2[:, 1] = -l/ 2
    w2[:, 2] = h / 2
    w2[:, 3] = i
    w3 = np.zeros(shape=(point_num, 4))
    w3[:, 0] = w_vector
    w3[:, 1] = l / 2
    w3[:, 2] = -h / 2
    w3[:, 3] = i
    w4 = np.zeros(shape=(point_num, 4))
    w4[:, 0] = w_vector
    w4[:, 1] =l / 2
    w4[:, 2] = h / 2
    w4[:, 3] = i
    l1 = np.zeros(shape=(point_num, 4))
    l1[:, 0] = -w / 2
    l1[:, 1] = l_vector
    l1[:, 2] = -h / 2
    l1[:, 3] = i
    l2 = np.zeros(shape=(point_num, 4))
    l2[:, 0] = -w / 2
    l2[:, 1] = l_vector
    l2[:, 2] = h / 2
    l2[:, 3] = i
    l3 = np.zeros(shape=(point_num, 4))
    l3[:, 0] = w / 2
    l3[:, 1] = l_vector
    l3[:, 2] = -h / 2
    l3[:, 3] = i
    l4 = np.zeros(shape=(point_num, 4))
    l4[:, 0] = w / 2
    l4[:, 1] = l_vector
    l4[:, 2] = h / 2
    l4[:, 3] = i

    point_mat=np.mat(np.concatenate((z1,z2,z3,z4,w1,w2,w3,w4,l1,l2,l3,l4,d1,d2,d3,d4)))#

    angle=point[6]-PI/2

    if pose is None:
        convert_mat = np.mat([[math.cos(angle), -math.sin(angle), 0, x],
                              [math.sin(angle), math.cos(angle), 0, y],
                              [0, 0, 1, z],
                              [0, 0, 0, label]])

        transformed_mat = convert_mat * point_mat.T
    else:

        convert_mat = np.mat([[math.cos(angle), -math.sin(angle), 0, 0],
                              [math.sin(angle), math.cos(angle), 0, 0],
                              [0, 0, 1, 0],
                              [0, 0, 0, 1]])
        transformed_mat = convert_mat * point_mat.T
        pose_mat = np.mat([[pose[0, 0], pose[0, 1], pose[0, 2], x],
                           [pose[1, 0], pose[1, 1], pose[1, 2], y],
                           [pose[2, 0], pose[2, 1], pose[2, 2], z],
                           [0, 0, 0, label]])
        transformed_mat = pose_mat * transformed_mat


    transformed_mat = np.array(transformed_mat.T,dtype=np.float32)

    return transformed_mat

In [ ]:
# Function to generate random colors based on color map
# Input: color map name
# Output: List of random colors
def generate_objects_color_map(color_map_name='rainbow'):
    color_map = []
    np.random.seed(4)

    x = 0
    for i in range(10000):
        if x > 1:
            x = np.random.random() * 0.5
        color_map.append(x)
        x += 0.2
    cmp = plt.get_cmap(color_map_name)
    color_map = cmp(color_map)
    color_map = color_map[:, 0:3] * 255
    color_map = color_map.astype(np.int).tolist()
    return color_map

# Function to map object indices to colors
# Input: Object indices array
# Input: Colors to select from
# Output: Color list
def generate_objects_colors(object_ids,color_map_list):
    assert len(color_map_list)>len(object_ids), "the color map list must longer than object indices list !"

    if len(object_ids)==0:
        return []
    else:
        colors=[]
        for i in object_ids:
            colors.append(color_map_list[i])
        return colors
    

# Function to map scatter to colors
# Input: Scatter points array
# Input: Colors to select from
# Output: Color list
def generate_scatter_colors(scatters,color_map_name='rainbow'):
    """
    map the scatters to colors
    :param scatters: (array or list(N,)),
    :param color_map_name: (str), the name of objects color map, such as "rainbow", "viridis","brg","gnuplot","hsv"
                             reference  https://matplotlib.org/stable/tutorials/colors/colormaps.html
    :return: (array(N,4)), each item represents (red, green, blue, alpha),
    """
    if len(scatters)==0:
        return []
    else:
        scatters = np.array(scatters)
        scatters_max = scatters.max()
        scatters_min = scatters.min()

        div = scatters_max-scatters_min
        if div !=0:
            scatters = (scatters-scatters_min)/div
        cmp = plt.get_cmap(color_map_name)
        new_colors = cmp(scatters)
        new_colors = new_colors[:, 0:3] * 255
        alpha = np.ones(shape=(len(new_colors), 1)) * 255
        new_colors = np.concatenate([new_colors, alpha], -1)
        
    return new_colors.astype(np.int)

In [ ]:
# Viewer Class containing all visualization functions
class Viewer:
    # Initialization Function
    # OpenPCDet: (x, y, z, l, w, h, yaw)
    def __init__(self,box_type = "OpenPCDet",bg=(255, 255, 255)):
        self.objects_color_map = generate_objects_color_map('rainbow')
        self.box_type = box_type
        self.vi = Plotter(bg=bg)
        self.set_lights()

        # data for rendering in 3D scene
        self.actors = []
        self.actors_without_del = []
        self.tracks_actors_dict = {}

        # data for rendering in 2D scene
        self.cam_intrinsic_mat = None
        self.cam_extrinsic_mat = None
        self.boxes_info = [] # (boxes:array(N,7), ids:array(N,), colors:array(N,3) or str, box_info:list(N,))
        self.points_info = [] # (boxes:array(N,3), colors:array(N,3) or str)
        self.image = None

    # Function to set lighting for vedo visualization
    def set_lights(self):
        def get_light(pos=(0, 0, 0), focalPoint=(0, 0, 0)):
            light = vtk.vtkLight()

            light.SetPosition(pos)
            light.SetFocalPoint(focalPoint)
            light.SetIntensity(0.65)

            return light

        light_actors = []

        light_actors.append(get_light(pos=(400, 400, 100), focalPoint=(0, 0, 0)))
        light_actors.append(get_light(pos=(-400, 400, 100), focalPoint=(0, 0, 0)))
        light_actors.append(get_light(pos=(400, -400, 100), focalPoint=(0, 0, 0)))
        light_actors.append(get_light(pos=(-400, -400, 100), focalPoint=(0, 0, 0)))
        for a in light_actors:
            self.vi.renderer.AddLight(a)


    # Function to set objects colors map
    # Returns a list of random colors
    def set_ob_color_map(self,color_map_name='rainbow'):
        self.objects_color_map = generate_objects_color_map(color_map_name)
        return self.objects_color_map

    # Function to load ego car model
    def set_ego_car(self,ego_car_path = "./ego_car.3ds"):
        ego_car = load(ego_car_path)
        ego_car.pos(-0.5, 0, -1.6)
        ego_car.scale(0.9)
        self.actors_without_del+=[ego_car]

    # Function to set intrinsic camera matrix
    def set_intrinsic_mat(self,intrinsic_mat):
        self.cam_intrinsic_mat = intrinsic_mat

    # Function to set extrinsic camera matrix
    def set_extrinsic_mat(self,extrinsic_mat):
        self.cam_extrinsic_mat = extrinsic_mat

    # Function to add points to visualization
    def add_points(self,points,
                   radius = 2,
                   color = (150,150,150),
                   scatter_filed=None,
                   alpha=1,
                   del_after_show='True',
                   add_to_3D_scene = True,
                   add_to_2D_scene = True,
                   color_map_name = "rainbow"):
        if scatter_filed is not None:
            colors = generate_scatter_colors(scatter_filed,color_map_name=color_map_name)
        else:
            colors = color

        if add_to_2D_scene:
            self.points_info.append((points,colors))

        if add_to_3D_scene:
            if del_after_show:
                self.actors.append(Points(points,r=radius,c=colors,alpha=alpha))
            else:
                self.actors_without_del.append(Points(points,r=radius,c=colors,alpha=alpha))

    # Function to add sphere to visualization
    def add_spheres(self,points,
                    radius = 0.3,
                    color='red',
                    res=30,
                    scatter_filed=None,
                    alpha=0.5,
                    del_after_show='True'):
        if scatter_filed is not None:
            colors = generate_scatter_colors(scatter_filed)[:,:3]
        else:
            colors = color

        if del_after_show:
            self.actors.append(Spheres(points,r=radius,res=res,c=colors,alpha=alpha))
        else:
            self.actors_without_del.append(Spheres(Points,r=radius,res=res,c=colors,alpha=alpha))

    # Function to add 3D Boxes to visualization
    def add_3D_boxes(self,boxes=None,
                     ids=None,
                     box_info=None,
                     color="blue",
                     add_to_3D_scene=True,
                     mesh_alpha = 0,
                     show_corner_spheres = True,
                     corner_spheres_alpha = 1,
                     corner_spheres_radius=0.1,
                     show_heading = True,
                     heading_scale = 1,
                     show_lines = True,
                     line_width = 2,
                     line_alpha = 1,
                     show_ids = True,
                     show_box_info=True,
                     del_after_show=True,
                     add_to_2D_scene=True,
                     caption_size=(0.05,0.05)
                     ):
        if boxes is None:
            return
        boxes= convert_box_type(boxes,self.box_type)
        if boxes is None:
            return

        if ids is not None:
            colors = generate_objects_colors(ids,self.objects_color_map)
        else:
            colors = color

        if add_to_2D_scene:
            self.boxes_info.append((boxes,ids,colors,box_info))

        if add_to_3D_scene:
            if del_after_show:
                self.actors += get_mesh_boxes(boxes,
                                              colors,
                                              mesh_alpha,
                                              ids,
                                              show_ids,
                                              box_info,
                                              show_box_info,
                                              caption_size)
                self.actors += get_line_boxes(boxes,
                                              colors,
                                              show_corner_spheres,
                                              corner_spheres_alpha,
                                              corner_spheres_radius,
                                              show_heading,
                                              heading_scale,
                                              show_lines,
                                              line_width,
                                              line_alpha)
            else:
                self.actors_without_del += get_mesh_boxes(boxes,
                                                          colors,
                                                          mesh_alpha,
                                                          ids,
                                                          show_ids,
                                                          box_info,
                                                          show_box_info,
                                                          caption_size)
                self.actors_without_del += get_line_boxes(boxes,
                                                          colors,
                                                          show_corner_spheres,
                                                          corner_spheres_alpha,
                                                          corner_spheres_radius,
                                                          show_heading,
                                                          heading_scale,
                                                          show_lines,
                                                          line_width,
                                                          line_alpha)

    # Function to add 3D car objects to visualization
    def add_3D_cars(self,boxes=None,
                     ids=None,
                     box_info=None,
                     color="blue",
                     mesh_alpha = 0.1,
                     show_ids = False,
                     show_box_info=False,
                     del_after_show=True,
                     car_model_path="./car.obj",
                     caption_size = (0.1, 0.1)
                    ):

        if boxes is None:
            return
        
        boxes= convert_box_type(boxes,self.box_type)
        if boxes is None:
            return

        if ids is not None:
            colors = generate_objects_colors(ids,self.objects_color_map)
        else:
            colors = color

        for i in range(len(boxes)):
            bb = boxes[i]
            size = bb[3:6]

            ang=bb[6]
            ang = int(ang / (2 * np.pi) * 360)

            if type(colors) is str:
                color = colors
            else:
                color = colors[i]

            if ids is not None:
                ob_id = ids[i]
                if ob_id in self.tracks_actors_dict.keys():
                    previous_ori=self.tracks_actors_dict[ob_id].GetOrientation()[2]
                    self.tracks_actors_dict[ob_id].pos(0,0,0)
                    self.tracks_actors_dict[ob_id].rotateZ(ang-previous_ori)
                    self.tracks_actors_dict[ob_id].pos(bb[0], bb[1], bb[2])

                    info = ""
                    if ids is not None and show_ids:
                        info = "ID: " + str(ids[i]) + '\n'
                    if box_info is not None and show_box_info:
                        info += str(box_info[i])
                    if info != '':
                        self.tracks_actors_dict[ob_id].caption(info,
                                                               point=(bb[0], bb[1] - bb[4] / 2, bb[2] + bb[5] / 2),
                                                               size=caption_size,
                                                               alpha=1,
                                                               c=color,
                                                               font="Calco",
                                                               justify='left')
                        self.tracks_actors_dict[ob_id]._caption.SetBorder(False)
                        self.tracks_actors_dict[ob_id]._caption.SetLeader(False)

                    if del_after_show:
                        self.actors.append(self.tracks_actors_dict[ob_id])
                    else:
                        self.actors_without_del.append(self.tracks_actors_dict[ob_id])
                else:

                    new_car=load(car_model_path)
                    new_car.scale((0.12,0.3,0.3))

                    new_car.scale(size)
                    new_car.rotateZ(ang)
                    new_car.pos(bb[0], bb[1], bb[2])

                    new_car.c(color)
                    new_car.alpha(mesh_alpha)
                    self.tracks_actors_dict[ob_id]=new_car
                    info = ""
                    if ids is not None and show_ids:
                        info = "ID: " + str(ids[i]) + '\n'
                    if box_info is not None and show_box_info:
                        info += str(box_info[i])
                    if info != '':
                        self.tracks_actors_dict[ob_id].caption(info,
                                                               point=(bb[0], bb[1] - bb[4] / 2, bb[2] + bb[5] / 2),
                                                               size=caption_size,
                                                               alpha=1,
                                                               c=color,
                                                               font="Calco",
                                                               justify='left')
                        self.tracks_actors_dict[ob_id]._caption.SetBorder(False)
                        self.tracks_actors_dict[ob_id]._caption.SetLeader(False)

                    if del_after_show:
                        self.actors.append(self.tracks_actors_dict[ob_id])
                    else:
                        self.actors_without_del.append(self.tracks_actors_dict[ob_id])

            else:
                new_car = load(car_model_path)
                new_car.scale((0.12, 0.3, 0.3))

                new_car.scale(size)
                new_car.rotateZ(ang)
                new_car.pos(bb[0], bb[1], bb[2])

                new_car.c(color)
                new_car.alpha(mesh_alpha)

                info = ""

                if box_info is not None and show_box_info:
                    info += str(box_info[i])
                if info != '':
                    new_car.caption(info,
                                   point=(bb[0], bb[1] - bb[4] / 2, bb[2] + bb[5] / 2),
                                   size=caption_size,
                                   alpha=1,
                                   c=color,
                                   font="Calco",
                                   justify='cent')
                    new_car._caption.SetBorder(False)
                    new_car._caption.SetLeader(False)
                if del_after_show:
                    self.actors.append(new_car)
                else:
                    self.actors_without_del.append(new_car)

    # Function to add image to visualzation
    def add_image(self,im):
        self.image = im
        return

    # Function to show 3D scene visualization
    def show_3D(self):
        self.vi.show(self.actors+self.actors_without_del,resetcam=False)
        self.vi.clear()
        self.actors.clear()
        self.points_info.clear()
        self.boxes_info.clear()

    # Function to show 2D scene visualization
    def show_2D(self,box_color = (255,0,0),show_box_info=False,show_ids=True,points_colors=(0,0,255)):
        if (self.cam_extrinsic_mat is None) or (self.cam_intrinsic_mat is None) or (self.image is None):
            return

        H,W,_ = self.image.shape

        for info in self.boxes_info:
            boxes, ids, colors, box_info=info

            if boxes is None:
                continue
            elif len(boxes) == 0:
                continue
            else:

                for box_id in range(len(boxes)):
                    box = boxes[box_id]
                    if type(colors) is not str:
                        color = [colors[box_id][2],colors[box_id][1],colors[box_id][0]]
                    else:
                        color = box_color

                    pts_3d_cam = get_box_points(box)
                    pts_3d_cam = velo_to_cam(pts_3d_cam[:,0:3],self.cam_extrinsic_mat)

                    img_pts = np.matmul(pts_3d_cam, self.cam_intrinsic_mat.T)  # (N, 3)
                    x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]

                    x = np.clip(x, 2, W-2)
                    y = np.clip(y, 2, H-2)

                    x = x.astype(np.int)
                    y = y.astype(np.int)

                    self.image[y, x] = color

                    x2 = x + 1
                    self.image[y, x2] = color
                    y2 = y + 1
                    self.image[y2, x] = color
                    self.image[y2, x2] = color

                    info = ""
                    if ids is not None and show_ids:
                        info +=  str(ids[box_id])+" "
                    if box_info is not None and show_box_info:
                        info += str(box_info[box_id])

                    if info != '':

                        text = info
                        org = ((max(x) - min(x)) // 2 + min(x), min(y) - 5)
                        fontFace = cv2.FONT_HERSHEY_DUPLEX
                        fontScale = 0.7
                        fontcolor = color  # BGR
                        thickness = 1
                        lineType = 4
                        cv2.putText(self.image, text, org, fontFace, fontScale, fontcolor, thickness, lineType)

        for points,colors in self.points_info:

            if type(colors) is tuple:

                color = [colors[2],colors[1],colors[0]]
            else:
                color = points_colors

            pts_3d_cam = velo_to_cam(points[:, 0:3], self.cam_extrinsic_mat)

            img_pts = np.matmul(pts_3d_cam, self.cam_intrinsic_mat.T)  # (N, 3)
            x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]

            x = np.clip(x, 2, W - 2)
            y = np.clip(y, 2, H - 2)

            x = x.astype(np.int)
            y = y.astype(np.int)

            self.image[y, x] = color

        cv2.imshow('im',self.image)
        cv2.waitKey(10)
        self.points_info.clear()
        self.boxes_info.clear()
        
    # Function to save 2D visualization
    def save_2D(self, image_file, box_color = (255,0,0),show_box_info=False,show_ids=True,points_colors=(0,0,255)):
        if (self.cam_extrinsic_mat is None) or (self.cam_intrinsic_mat is None) or (self.image is None):
            return

        H,W,_ = self.image.shape

        for info in self.boxes_info:
            boxes, ids, colors, box_info=info

            if boxes is None:
                continue
            elif len(boxes) == 0:
                continue
            else:

                for box_id in range(len(boxes)):
                    box = boxes[box_id]
                    if type(colors) is not str:
                        color = [colors[box_id][2],colors[box_id][1],colors[box_id][0]]
                    else:
                        color = box_color

                    pts_3d_cam = get_box_points(box)
                    pts_3d_cam = velo_to_cam(pts_3d_cam[:,0:3],self.cam_extrinsic_mat)

                    img_pts = np.matmul(pts_3d_cam, self.cam_intrinsic_mat.T)  # (N, 3)
                    x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]

                    x = np.clip(x, 2, W-2)
                    y = np.clip(y, 2, H-2)

                    x = x.astype(np.int)
                    y = y.astype(np.int)

                    self.image[y, x] = color

                    x2 = x + 1
                    self.image[y, x2] = color
                    y2 = y + 1
                    self.image[y2, x] = color
                    self.image[y2, x2] = color

                    info = ""
                    if ids is not None and show_ids:
                        info +=  str(ids[box_id])+" "
                    if box_info is not None and show_box_info:
                        info += str(box_info[box_id])

                    if info != '':

                        text = info
                        org = ((max(x) - min(x)) // 2 + min(x), min(y) - 5)
                        fontFace = cv2.FONT_HERSHEY_DUPLEX
                        fontScale = 0.7
                        fontcolor = color  # BGR
                        thickness = 1
                        lineType = 4
                        cv2.putText(self.image, text, org, fontFace, fontScale, fontcolor, thickness, lineType)

        for points,colors in self.points_info:
            if isinstance(colors,(list,tuple,np.ndarray)):
                color = []
                for c in colors:
                    color.append([c[2], c[1], c[0]])
            else:
                color = points_colors

            pts_3d_cam = velo_to_cam(points[:, 0:3], self.cam_extrinsic_mat)

            img_pts = np.matmul(pts_3d_cam, self.cam_intrinsic_mat.T)  # (N, 3)
            x, y = img_pts[:, 0] / img_pts[:, 2], img_pts[:, 1] / img_pts[:, 2]

            x = np.clip(x, 2, W - 2)
            y = np.clip(y, 2, H - 2)

            x = x.astype(np.int)
            y = y.astype(np.int)
            
            self.image[y, x] = color

        print(f"Saving Image: {image_file}")
        cv2.imwrite(image_file, self.image)
        self.points_info.clear()
        self.boxes_info.clear()
        
    # Function to create birds eye view of lidar
    def birds_eye_view(self, image_file, box_color = (255,0,0),show_box_info=False,
                       show_ids=True,points_colors=(0,0,255)):
        if (self.cam_extrinsic_mat is None) or (self.cam_intrinsic_mat is None) or (self.image is None):
            return

        H,W,_ = self.image.shape

        for info in self.boxes_info:
            boxes, ids, colors, box_info=info

            if boxes is None:
                continue
            elif len(boxes) == 0:
                continue
            else:

                for box_id in range(len(boxes)):
                    box = boxes[box_id]
                    if type(colors) is not str:
                        color = [colors[box_id][2],colors[box_id][1],colors[box_id][0]]
                    else:
                        color = box_color

                    pts_3d_cam = get_box_points(box)
                    
                    # Extract points
                    x_points = pts_3d_cam[:, 0]
                    y_points = pts_3d_cam[:, 1]

                    # Convert to pixel positions
                    x_img = x_points
                    y_img = y_points

                    x_img = (x_img - np.min(x_img)) * W / np.ptp(x_img)
                    y_img = (y_img - np.min(y_img)) * H / np.ptp(y_img)

                    # FILL PIXEL VALUES IN IMAGE ARRAY
                    x_img = np.clip(x_img, 2, W - 2)
                    y_img = np.clip(y_img, 2, H - 2)

                    x = x_img.astype(np.int)
                    y = y_img.astype(np.int)

                    self.image[y, x] = color

                    info = ""
                    if ids is not None and show_ids:
                        info +=  str(ids[box_id])+" "
                    if box_info is not None and show_box_info:
                        info += str(box_info[box_id])

                    if info != '':

                        text = info
                        org = ((max(x) - min(x)) // 2 + min(x), min(y) - 5)
                        fontFace = cv2.FONT_HERSHEY_DUPLEX
                        fontScale = 0.7
                        fontcolor = color  # BGR
                        thickness = 1
                        lineType = 4
                        cv2.putText(self.image, text, org, fontFace, fontScale, fontcolor, thickness, lineType)

        for points,colors in self.points_info:
            if isinstance(colors,(list,tuple,np.ndarray)):
                color = []
                for c in colors:
                    color.append([c[2], c[1], c[0]])
            else:
                color = points_colors
                
            pts_3d_cam = points
                            
            # Extract points
            x_points = pts_3d_cam[:, 0]
            y_points = pts_3d_cam[:, 1]

            # Convert to pixel positions
            x_img = x_points
            y_img = y_points
            
            x_img = (x_img - np.min(x_img)) * W / np.ptp(x_img)
            y_img = (y_img - np.min(y_img)) * H / np.ptp(y_img)

            # FILL PIXEL VALUES IN IMAGE ARRAY
            x_img = np.clip(x_img, 2, W - 2)
            y_img = np.clip(y_img, 2, H - 2)

            x_img = x_img.astype(np.int)
            y_img = y_img.astype(np.int)
            self.image[y_img, x_img] = color

        print(f"Saving Image: {image_file}")
        cv2.imwrite(image_file, self.image)
        self.points_info.clear()
        self.boxes_info.clear()

## 4. Visualise
Combine all the code developed above

In [ ]:
root = '../input/kitti-3d-object-detection-dataset/training'
label_path = '../input/kitti-3d-object-detection-dataset/training/label_2'

dataset = KittiDetectionDataset(root, label_path)

vi = Viewer(box_type = "Kitti")
vi.set_ob_color_map('gnuplot')

def visualize_points(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    black_image = np.zeros_like(image)
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    ids = labels[:, -1].astype(int)
    
    vi.add_image(black_image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.add_points(points[:,0:3], radius = 1000, color = (150,150,150),
                  scatter_filed = points[:,2], alpha=0, del_after_show = True,
                  add_to_3D_scene = True, add_to_2D_scene = True,
                  color_map_name = "cool")
    vi.save_2D(f"point_image_{index}.png")
    
def visualize_bev(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    black_image = np.zeros_like(image)
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    
    vi.add_image(black_image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.add_points(points[:,0:3], radius = 1000, color = (150,150,150),
                  scatter_filed = points[:,2], alpha=0, del_after_show = True,
                  add_to_3D_scene = True, add_to_2D_scene = True,
                  color_map_name = "cool")
    vi.birds_eye_view(f"bev_image_{index}.png")
    
def visualize_bev_box(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    black_image = np.zeros_like(image)
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    ids = labels[:, -1].astype(int)
    
    vi.add_image(black_image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.add_points(points[:,0:3], radius = 1000, color = (150,150,150),
                  scatter_filed = points[:,2], alpha=0, del_after_show = True,
                  add_to_3D_scene = True, add_to_2D_scene = True,
                  color_map_name = "cool")
    vi.add_3D_boxes(boxes=labels, ids=ids, box_info=label_names,
                 color="blue", add_to_3D_scene=True, add_to_2D_scene = True,
                 mesh_alpha = 0.3, show_corner_spheres = True, corner_spheres_alpha = 1,
                 corner_spheres_radius=0.1, show_heading = True, heading_scale = 1,
                 show_lines = True, line_width = 2, line_alpha = 1, show_ids = True,
                 show_box_info=True, del_after_show=True, caption_size=(0.05,0.05))
    
    vi.birds_eye_view(f"bev_image_{index}.png")
    
def visualize_image(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    
    vi.add_image(image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.save_2D(f"image_{index}.png")
    
def visualize_image_box(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    ids = labels[:, -1].astype(int)
    
    vi.add_image(image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.add_3D_boxes(boxes=labels, ids=ids, box_info=label_names,
                 color="blue", add_to_3D_scene=True, add_to_2D_scene = True,
                 mesh_alpha = 0.3, show_corner_spheres = True, corner_spheres_alpha = 1,
                 corner_spheres_radius=0.1, show_heading = True, heading_scale = 1,
                 show_lines = True, line_width = 2, line_alpha = 1, show_ids = True,
                 show_box_info=True, del_after_show=True, caption_size=(0.05,0.05))
    vi.save_2D(f"image_box_{index}.png")

def visualize(index):
    P2, V2C, points, image, labels, label_names = dataset[index]
    white_image = np.ones_like(image) * 255
    
    mask = label_names=="Car"
    labels = labels[mask]
    label_names = label_names[mask]
    ids = labels[:, -1].astype(int)
    
    vi.add_image(white_image)
    vi.set_extrinsic_mat(V2C)
    vi.set_intrinsic_mat(P2)
    vi.add_points(points[:,0:3], radius = 2, color = (150,150,150),
                  scatter_filed = points[:,2], alpha=0, del_after_show = True,
                  add_to_3D_scene = True, add_to_2D_scene = True,
                  color_map_name = "viridis")
    
    vi.add_3D_boxes(boxes=labels, ids=ids, box_info=label_names,
                 color="blue", add_to_3D_scene=True, add_to_2D_scene = True,
                 mesh_alpha = 0.3, show_corner_spheres = True, corner_spheres_alpha = 1,
                 corner_spheres_radius=0.1, show_heading = True, heading_scale = 1,
                 show_lines = True, line_width = 2, line_alpha = 1, show_ids = True,
                 show_box_info=True, del_after_show=True, caption_size=(0.05,0.05))
    
    vi.save_2D(f'image_full_{index}.png')

In [ ]:
index_list = random.sample(range(len(dataset)), 5)

def show_images(func, prefix):
    images = []
    for index in index_list:
        func(index)
        images.append(mpimg.imread(prefix + f"{index}.png"))
        
    for image in images:
        plt.figure(figsize=(30, 30), dpi=80)
        plt.imshow(image)
        plt.show()

In [ ]:
show_images(visualize_points, "point_image_")

In [ ]:
show_images(visualize_bev, "bev_image_")

In [ ]:
show_images(visualize_image, "image_")

In [ ]:
show_images(visualize_image_box, "image_box_")

In [ ]:
show_images(visualize, "image_full_")